# Numpy2AD Tutorial
## Generating Adjoint Code for Numpy Matrix Expressions

In this tutorial notebook we will present some basic examples on how to use **Numpy2AD**.

The motivation for this package is the following: matrix expressions can be understood high-dimensional functions. For example, a simple **matrix-vector product** takes $ n \times m$ (the matrix) plus $m$ (the vector) **inputs** to compute $n$ **outputs**. We are interested in the partial derivatives of **each output** with respect to **all inputs** (the Jacobian tensor). Naive [finite differences](https://en.wikipedia.org/wiki/Finite_difference_method) would require us to perform as many "forward" passes (expression evaluations) as there are input entries. In [adjoint (reverse) mode algorithmic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation#Reverse_accumulation), **only $n$** combined forward and backwards passes are needed. Clearly, $O(n)$ scales much better than $O(mn + m)$ for larger and larger problem sizes. 

Numpy2AD applies the method of **source-to-source** transformation to generate adjoint code for matrix expressions that contain any permutation of **addition, multiplication, transposing,** and **inverting**. By using this package you can benefit from transparently **readable and understandable** derivative code for your sensitivity analysis and optimzation problems. The generated code can be easily pasted into a script or imported as a module. Importantly, there is no runtime overhead compared to other autodiff frameworks, e.g. jax or pytorch, and no new syntax to learn.

In [ ]:
import ast
import numpy as np
from numpy2ad import draw_AST, transform, transform_expr

## 1st Example: Transforming an Expression

We use the `@` operator as shorthand for m**at**mul (matrix multiplication). This allows for a more natural, math-like syntax.

First, let us explore the expression tree, also referred to as *abstract syntax tree* (AST), of a simple **matrix-matrix multiplication** and **addition**.

In [ ]:
# Dump the native ast
print(ast.dump(ast.parse("D = A @ B + C"), indent=3))

As you can see, the expression is a nested structure essentially composed of one *Assign* and two *BinOp*s (binary operations).

With Numpy2AD's `draw_AST` we can get a graphical representation.

In [ ]:
# Dot graph for AST
draw_AST("D = A @ B + C")

Now let us transform the expression to adjoint code.

Essentially, the AST is first *linearized* into *single-assignment code* by introducing intermediate variables `v{i}`. Partial derivatives of each assignment are then generated and appended in reverse order.

In [ ]:
print(transform_expr("Y = A @ B + C"))

Note how adjoints (suffix `_a`) are always incremented to allow for correct derivative propagation.

To use this code we must import numpy and correctly seed the adjoint variables. Below we calculate the adjoint of the first entry in `Y`. Remember that accumulating the full Jacobian will require a loop over all outputs (not shown here).

In [ ]:
dims = (3, 3)
A = np.ones(dims)
A_a = np.zeros(dims)  # dYdA
B = np.full(dims, 2.0)
B_a = np.zeros(dims)  # dYdB
C = np.full(dims, 3.0)
C_a = np.zeros(dims)  # dYdC
Y_a = np.zeros(dims)
Y_a[0, 0] = 1  # adjoint seed

# adjoint code
v0 = A @ B
Y = v0 + C
v0_a = np.zeros(v0.shape)
C_a += Y_a
v0_a += Y_a
B_a += A.T @ v0_a
A_a += v0_a @ B.T

In [ ]:
print(f"Y =\n {Y}\ndY/dA = \n {A_a}\ndY/dB =\n {B_a}\ndY/dC =\n {C_a}")

The results can be easily verified: the first entry in `Y` is, by the rules of matrix-matrix products, the inner product of the first **row** of `A` and the first **column** of `B` plus the first entry in `C`. Therefore, the partial derivatives with respect to the input matrices are nonzero in exactly those entries.

In [ ]:
# AST of transformed expression
draw_AST(transform_expr("D = A @ B + C"))

## 2nd Example: Transforming a Function

We define a simple Python function `simple_func` that computes the same matrix expression as above and apply Numpy2AD's `transform` method to it.

In [ ]:
def simple_func(A, B, C):
    return A @ B + C

In [ ]:
simple_func_transformed = transform(simple_func)
print(simple_func_transformed)

The result is a string in which the corresponding *adjoint function* is defined. 

Some things to note here:
- The function name is automatically extended with `_ad` and adjoints of function arguments (`A_a`, `B_a`, ...) and output (`out_a`) are appended.
- The result of the original function is returned as `out` in the first entry of the return tuple. All adjoint varibles follow.
- Adjoints of all intermediate variables `v{i}` are zero-initialized.
- Numpy is automatically imported.

The `transform` method has an optional argument called `out_file` with which the adjoint function can be conveniently exported as a Python module. If you wish to immediately use the function, it must first be compiled to a code object, as shown below.

In [ ]:
import numpy as np

# Compile and execute it to make it visible
exec(compile(simple_func_transformed, filename="<ast>", mode="exec"))

# active arguments
dims = (3, 3)
A = 1.5 * np.ones(dims)
B = 2.0 * np.ones(dims)
C = 3.0 * np.ones(dims)
# initialize adjoints
A_a = np.zeros(dims)
B_a = np.zeros(dims)
C_a = np.zeros(dims)
Y_a = np.zeros(dims)
# seed the adjoint direction
Y_a[0][0] = 1

result, A_a, B_a, C_a = simple_func_ad(A, B, C, A_a, B_a, C_a, Y_a)
print(f"primal result:\n {result}\n dy/dA:\n {A_a}\n dy/dB:\n {B_a}\n dy/dC:\n {C_a}")

## 3rd Example: A Real Use Case

Consider the [*Generalized Least Squares*](https://en.wikipedia.org/wiki/Generalized_least_squares) problem (GLS). The goal is to estimate regression coefficients $b$ from observations $y$ with design matrix $X$ and error covariance matrix $M$. The derivatives will tells us how e.g. the coefficients depends on the observations (uncertain) or model parameters.

We choose a simple linear model: $y_i = \beta_0 + \beta_1 \, x_i$ with measurement error covariance: $\sigma_i^2 = 0.01 + \alpha \, x_i$. For $\alpha \neq 0$ the problem become *heteroscedastic*, essentially meaning that the uncertainty increases proportionally to the magnitude of $x$.

Below is an implementation of the closed-form solution to the GLS problem and its corresponding adjoint code for sensitivity analysis.

In [ ]:
def GLS(X, M, y):
    M_inv = np.linalg.inv(M)
    return np.linalg.inv(X.T @ M_inv @ X) @ X.T @ M_inv @ y

GLS_ad_str = transform(GLS)
print(GLS_ad_str)

exec(compile(GLS_ad_str, filename="<ast>", mode="exec"))

### Noise-free Case

We are interested in the sensitivity (partial derivatives) of the slope parameter `b[1]` with respect to all measurements. This will tell us how "important" individual data points are.

In [ ]:
import matplotlib.pyplot as plt

N = 100
K = 2

# Construct the covariance matrix
def build_system(alpha):
    x = np.linspace(0, 1, N)  # x from 0 to 1
    var = 0.01 + alpha * x # variances
    cov_mat = np.zeros((N, N))
    np.fill_diagonal(cov_mat, var)  # no cross-correlations terms
    samples = (x + np.random.multivariate_normal(
        mean=np.zeros(N), cov=cov_mat)).reshape((N, 1))
    return x, samples, cov_mat

# plot measurements and ground truth
x, samples, cov_mat = build_system(alpha=0.0)

plt.figure(figsize=(5, 5))
plt.plot(x, samples, ".")
plt.plot(x, x, "r--", label="y=x")
plt.xlabel("$x$")
plt.ylabel("$y(x)$")
plt.legend();

In [ ]:
# Initialize
y = samples
y_a = np.zeros_like(samples)
X = np.ones((N, K))
X[:, 1] = x
X_a = np.zeros_like(X)
M = cov_mat
M_a = np.zeros_like(M)

b_a = np.zeros((K, 1))
b_a[1] = 1  # differentiate w.r.t slope parameter of regression

b, X_a, M_a, y_a = GLS_ad(X, M, y, X_a, M_a, y_a, b_a)
print(b)  # should be close to [0, 1] (an ideal linear slope)

Now let us plot the entries in the gradient.

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(y_a, ".")
plt.xlabel("$i$")
plt.ylabel(r"$\nabla_y \, b_1(i)$")
plt.title("Gradient magnitude: " + f"{np.linalg.norm(y_a): .3f}");

We can see that "early" and "late" samples contribute equally. The zero-crossing happens at exactly the midpoint sample.

### Noisy Case

Now let us add some heteroscedastic noise to the measurements and see how the sensitivities change.

In [ ]:
x, samples, cov_mat = build_system(alpha=0.1)
plt.figure(figsize=(5, 5))
plt.plot(x, samples, ".")
plt.plot(x, x, "r--", label="y=x")
plt.xlabel("$x$")
plt.ylabel("$y(x)$")
plt.legend();

In [ ]:
# Initialize
y = samples
y_a = np.zeros(samples.shape)
X = np.ones((N, K))
X[:, 1] = x
X_a = np.zeros_like(X)
M = cov_mat
M_a = np.zeros_like(M)

b_a = np.zeros((K, 1))

b, X_a, M_a, y_a = GLS_ad(X, M, y, X_a, M_a, y_a, b_a)
print(b) 

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(y_a)
plt.xlabel("$i$")
plt.ylabel(r"$\nabla_y \, b_1(i)$")
plt.title("Gradient magnitude: " + f"{np.linalg.norm(y_a): .3f}");

Clearly, early samples contribute nonlinearly more than later ones! 

This insight to the GLS model might be intuitive to some but with Numpy2AD's adjoint transformation it can be clearly shown in code. There are many more effects to analyze here but that is beyond the scope of this tutorial.

If you have any questions, please feel free to reach out to us on GitHub.